In [200]:
#!/usr/bin/python
# coding: utf-8
import numpy as np
import pandas as pd
import geopandas as gpd
import scipy
import shapely
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import datetime
import time
import pickle
from shapely.ops import nearest_points
from shapely.geometry import Point, MultiPoint, Polygon, LineString


In [3]:
df = pd.read_csv('pop\\LOCAL_PEOPLE_201804\\LOCAL_PEOPLE_20180403.csv', encoding='utf-8')
import glob
for file in glob.glob("pop\\LOCAL_PEOPLE_201804\\*.csv")[1:]:
    df = pd.concat([df, pd.read_csv(file, encoding='utf-8')], ignore_index=True)

In [4]:
print(df.shape)
df.head()

(9192244, 33)


,기준일ID,시간대구분,행정동코드,집계구코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20180403,21,11215820,1105064010003,172.0,*,*,*,12,13,...,22,19,9,9,6,5,*,6,*,*
1,20180403,2,11350570,1111052010301,67.0,7,*,*,*,*,...,*,*,*,*,*,4,*,*,*,4
2,20180403,0,11350621,1111061020002,1.0,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
3,20180403,23,11350640,1111066040009,56.0,4,*,*,*,*,...,*,*,*,*,*,*,*,*,*,4
4,20180403,7,11350720,1111074010112,50.0,*,*,*,*,*,...,*,*,*,*,4,*,*,*,*,*


In [8]:
use_df = df[['기준일ID', '시간대구분', '행정동코드', '집계구코드', '총생활인구수', '남자65세부터69세생활인구수', '남자70세이상생활인구수']]

In [14]:
use_df['남자65세부터69세생활인구수'] = use_df['남자65세부터69세생활인구수'].replace('*', 0)
use_df['남자70세이상생활인구수'] = use_df['남자70세이상생활인구수'].replace('*', 0)


c:\programdata\anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\programdata\anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,기준일ID,시간대구분,행정동코드,집계구코드,총생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수
0,20180403,21,11215820,1105064010003,172.0,0,0
1,20180403,2,11350570,1111052010301,67.0,0,0
2,20180403,0,11350621,1111061020002,1.0,0,0
3,20180403,23,11350640,1111066040009,56.0,0,0
4,20180403,7,11350720,1111074010112,50.0,0,0


In [23]:
use_df['남자65세부터69세생활인구수'] = pd.to_numeric(use_df['남자65세부터69세생활인구수'], errors='coerce')
use_df['남자70세이상생활인구수'] = pd.to_numeric(use_df['남자70세이상생활인구수'])
use_df.head()

c:\programdata\anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\programdata\anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,기준일ID,시간대구분,행정동코드,집계구코드,총생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수
0,20180403,21,11215820,1105064010003,172.0,0,0
1,20180403,2,11350570,1111052010301,67.0,0,0
2,20180403,0,11350621,1111061020002,1.0,0,0
3,20180403,23,11350640,1111066040009,56.0,0,0
4,20180403,7,11350720,1111074010112,50.0,0,0


In [34]:
final = use_df.groupby(['시간대구분', '집계구코드']).agg({'총생활인구수': 'mean', '남자65세부터69세생활인구수': 'mean', '남자70세이상생활인구수': 'mean'})
final.head()

총생활인구수  남자65세부터69세생활인구수  남자70세이상생활인구수
시간대구분 집계구코드                                                
0     1101053010001   319.85             3.55         16.80
      1101053010002  1301.55            27.55         55.20
      1101053010003  1229.20            21.50         53.55
      1101053010004  1481.85            26.25         87.35
      1101053010005  1054.95            14.95         50.40

In [35]:
final.describe()

,총생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수
count,459672.000000,459672.000000,459672.000000
mean,580.022265,11.344566,24.602552
std,1190.350842,21.241185,42.695636
min,0.000000,0.000000,0.000000
25%,191.700000,2.300000,7.950000
50%,353.850000,7.100000,15.400000
75%,616.550000,13.100000,27.750000
max,71142.300000,812.100000,2239.550000


In [40]:
final = final.reset_index()

In [41]:
final.groupby('시간대구분').agg({'집계구코드': 'count'})

,집계구코드
시간대구분,
0,19153
1,19153
2,19153
3,19153
4,19153
5,19153
6,19153
7,19153
8,19153


In [42]:
final.head()

,시간대구분,집계구코드,총생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수
0,0,1101053010001,319.85,3.55,16.80
1,0,1101053010002,1301.55,27.55,55.20
2,0,1101053010003,1229.20,21.50,53.55
3,0,1101053010004,1481.85,26.25,87.35
4,0,1101053010005,1054.95,14.95,50.40


In [143]:
use_time_df = final.loc[final['시간대구분'] == 3]
#use_time_df = pd.concat([use_time_df, final.loc[final['시간대구분'] == 7]], ignore_index=True)

In [144]:
use_time_df.head()

,시간대구분,집계구코드,총생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수
57459,3,1101053010001,289.10,1.65,15.45
57460,3,1101053010002,1286.70,28.10,51.15
57461,3,1101053010003,1269.45,22.90,52.55
57462,3,1101053010004,1308.65,22.60,84.40
57463,3,1101053010005,936.10,12.60,48.60


In [115]:
use_time_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19153 entries, 268142 to 287294
Data columns (total 5 columns):
시간대구분              19153 non-null int64
집계구코드              19153 non-null int64
총생활인구수             19153 non-null float64
남자65세부터69세생활인구수    19153 non-null float64
남자70세이상생활인구수       19153 non-null float64
dtypes: float64(3), int64(2)
memory usage: 897.8 KB


In [116]:
fp_test = '집계구.shp'
area_gdf = gpd.read_file(fp_test, encoding='cp949')
print('before len :', len(area_gdf.index))
area_gdf = area_gdf.dropna(subset=['geometry'])
print('after len :', len(area_gdf.index))
print(area_gdf.crs)

before len : 19153
after len : 19153
{}


In [145]:
area_gdf['TOT_REG_CD'] = pd.to_numeric(area_gdf['TOT_REG_CD'])
area_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19153 entries, 0 to 19152
Data columns (total 4 columns):
TOT_REG_CD    19153 non-null int64
ADM_NM        19153 non-null object
ADM_CD        19153 non-null object
geometry      19153 non-null object
dtypes: int64(1), object(3)
memory usage: 748.2+ KB


In [146]:
home_df = pd.merge(area_gdf, use_time_df, left_on='TOT_REG_CD', right_on='집계구코드', how='left')
home_df.tail()

,TOT_REG_CD,ADM_NM,ADM_CD,geometry,시간대구분,집계구코드,총생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수
19148,1102073010501,동화동,1102073,"POLYGON ((957399.4255924225 1951569.49789238, ...",3,1102073010501,396.95,8.55,17.75
19149,1102055030001,명동,1102055,"POLYGON ((954867.1474895477 1952204.599294662,...",3,1102055030001,1035.75,35.05,41.60
19150,1102058040004,장충동,1102058,"POLYGON ((956138.5723896027 1951190.621294022,...",3,1102058040004,1748.10,40.10,48.00
19151,1121052010002,보라매동,1121052,"POLYGON ((949365.8065204639 1944046.501459125,...",3,1121052010002,1627.10,37.05,85.10
19152,1121052010001,보라매동,1121052,"POLYGON ((949249.8090229044 1943731.163557053,...",3,1121052010001,89.25,2.45,2.85


In [147]:
type(home_df)

geopandas.geodataframe.GeoDataFrame

In [148]:
home_df['sum'] = home_df['남자65세부터69세생활인구수'] + home_df['남자70세이상생활인구수']

In [149]:
home_df.head()

,TOT_REG_CD,ADM_NM,ADM_CD,geometry,시간대구분,집계구코드,총생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수,sum
0,1101053010006,사직동,1101053,"POLYGON ((953231.9682865143 1953435.865716934,...",3,1101053010006,766.45,12.00,42.50,54.50
1,1101053010001,사직동,1101053,"POLYGON ((952858.5655841827 1952976.274114609,...",3,1101053010001,289.10,1.65,15.45,17.10
2,1101053010003,사직동,1101053,"POLYGON ((952889.1776866913 1953405.245515823,...",3,1101053010003,1269.45,22.90,52.55,75.45
3,1101053010002,사직동,1101053,"POLYGON ((953093.8012866974 1953337.962316513,...",3,1101053010002,1286.70,28.10,51.15,79.25
4,1101053010005,사직동,1101053,"POLYGON ((953553.9324855804 1953335.741514206,...",3,1101053010005,936.10,12.60,48.60,61.20


In [151]:
home_df['seq'] = home_df.index

In [153]:
home_df.to_csv('time3_home_pop.csv', encoding='cp949', index=False)

In [150]:
home_df.to_file("time3_home_pop.shp", encoding='cp949')

In [206]:
home_df['ADM_CD'] = pd.to_numeric(home_df['ADM_CD'])
#code_list = [1106080, 1103053, 1106083, 1116065, 1107070, 1106081, 1109062, 1106082, 1111078, 1104071, 1124056, 1106091, 1111052, 1123075, 1111058, 1116054, 1112051, 1116066, 1106087, 1103070, 1107068, 1106084, 1109061, 1106072, 1118058, 1111064, 1119066, 1107054, 1106088, 1109073, 1125056, 1123074, 1125051, 1123073, 1111079, 1101061, 1111076, 1115057, 1115053, 1106086, 1106090, 1115059, 1110056, 1119073, 1102054, 1120063, 1106089, 1101067, 1111053, 1116072, 1107057, 1114069, 1114074]
#code_list = [1106080, 1106083, 1111078, 1106081, 1106082, 1116054, 1112051, 1111052, 1111076, 1116065, 1123075, 1109062, 1111058, 1103053, 1111079, 1106087, 1115070]
code_list = [1106081, 1106080, 1106083, 1106082, 1112051, 1103053, 1119066, 1101061]

In [389]:
target_df = target_df.loc[target_df['ADM_CD'].isin(code_list)]

In [390]:
target_df.shape

(320, 13)

In [209]:
target_df.to_file("target_time3_home_pop_w8.shp", encoding='cp949')


In [170]:
target_df.to_csv('target_time3_home_pop.csv', encoding='cp949', index=False)

In [175]:
old_df = pd.read_csv('old_pop.csv', encoding='cp949')
print(old_df.shape)
old_df.head()

(424, 24)


,ADM_CD,H_DNG_CD,DO_NM,CT_NM,ADM_NM,03시_생활인구,총인구,노령인구,합계_계,합계_남성,...,일반_계,일반_남성,저소득합,노인비,독거노인비,취약독거노인비,생활인구x취약독거노인비,Unnamed: 21,비합,비평
0,1106080,11230705,서울,동대문구,청량리동,1940.20,22562,4475,1603,614,...,473,58,1130,0.198342,0.358212,0.704928,15904.591390,0.050084,1.261483,0.420494
1,1103053,11170530,서울,용산구,남영동,853.85,7575,1239,852,515,...,298,139,554,0.163564,0.687651,0.650235,4925.528169,0.073135,1.501450,0.500483
2,1106083,11230560,서울,동대문구,전농1동,2588.35,30107,4347,1774,709,...,685,200,1089,0.144385,0.408098,0.613867,18481.692780,0.036171,1.166350,0.388783
3,1116065,11500604,서울,강서구,가양2동,1235.95,16502,2863,1028,219,...,402,34,626,0.173494,0.359064,0.608949,10048.883270,0.037935,1.141507,0.380502
4,1107070,11260690,서울,중랑구,신내2동,1615.20,23045,2895,704,172,...,297,26,407,0.125624,0.243178,0.578125,13322.890630,0.017661,0.946927,0.315642


In [176]:
old_df = old_df[['ADM_CD', '취약독거노인비']]

In [177]:
old_df.head()

,ADM_CD,취약독거노인비
0,1106080,0.704928
1,1103053,0.650235
2,1106083,0.613867
3,1116065,0.608949
4,1107070,0.578125


In [384]:
target_df = pd.merge(home_df, old_df, on='ADM_CD')

In [385]:
target_df.head()

,TOT_REG_CD,ADM_NM,ADM_CD,geometry,시간대구분,집계구코드,총생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수,sum,seq,취약독거노인비
0,1101053010006,사직동,1101053,"POLYGON ((953231.9682865143 1953435.865716934,...",3,1101053010006,766.45,12.00,42.50,54.50,0,0.143251
1,1101053010001,사직동,1101053,"POLYGON ((952858.5655841827 1952976.274114609,...",3,1101053010001,289.10,1.65,15.45,17.10,1,0.143251
2,1101053010003,사직동,1101053,"POLYGON ((952889.1776866913 1953405.245515823,...",3,1101053010003,1269.45,22.90,52.55,75.45,2,0.143251
3,1101053010002,사직동,1101053,"POLYGON ((953093.8012866974 1953337.962316513,...",3,1101053010002,1286.70,28.10,51.15,79.25,3,0.143251
4,1101053010005,사직동,1101053,"POLYGON ((953553.9324855804 1953335.741514206,...",3,1101053010005,936.10,12.60,48.60,61.20,4,0.143251


In [386]:
print(len(target_df.index))
target_df.dropna(subset=['취약독거노인비'])
len(target_df.index)

19153


19153

In [387]:
target_df['fin'] = target_df['sum'] * target_df['취약독거노인비']

In [388]:
target_df.head()

,TOT_REG_CD,ADM_NM,ADM_CD,geometry,시간대구분,집계구코드,총생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수,sum,seq,취약독거노인비,fin
0,1101053010006,사직동,1101053,"POLYGON ((953231.9682865143 1953435.865716934,...",3,1101053010006,766.45,12.00,42.50,54.50,0,0.143251,7.807163
1,1101053010001,사직동,1101053,"POLYGON ((952858.5655841827 1952976.274114609,...",3,1101053010001,289.10,1.65,15.45,17.10,1,0.143251,2.449587
2,1101053010003,사직동,1101053,"POLYGON ((952889.1776866913 1953405.245515823,...",3,1101053010003,1269.45,22.90,52.55,75.45,2,0.143251,10.808264
3,1101053010002,사직동,1101053,"POLYGON ((953093.8012866974 1953337.962316513,...",3,1101053010002,1286.70,28.10,51.15,79.25,3,0.143251,11.352617
4,1101053010005,사직동,1101053,"POLYGON ((953553.9324855804 1953335.741514206,...",3,1101053010005,936.10,12.60,48.60,61.20,4,0.143251,8.766942


In [205]:
target_df.to_file("target_time3_home_pop_all.shp", encoding='cp949')

In [196]:
target_df.to_csv('target_time3_home_pop_all.csv', encoding='cp949', index=False)

In [497]:
office_df = pd.read_csv('subway.csv', encoding='utf8')
office_df.tail(1)

,전철역코드,전철역명,호선,외부코드,사이버스테이션,X좌표,Y좌표,X좌표(WGS),Y좌표(WGS)
928,4510,명지대,E,Y118,4510,542100.0,1037900.0,37.237964,127.190294


In [435]:
office_df = pd.read_excel('서울특별시 보건의료원(소) 정보_20180821.xlsx', encoding='cp949')
print(office_df.shape)
office_df.tail(1)

(25, 23)


,번호,사업장명,소재지전체주소,도로명전체주소,영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,소재지면적,...,의료인수,입원실수,진료과목내용,진료과목내용명,총면적,총인원,전화번호,위치정보(X),위치정보(Y),상세영업상태명
20,21,서울특별시강서구보건소,서울특별시 강서구 염창동 275번지 12호,"서울특별시 강서구 공항대로 561 (염창동, 강서보건소)",운영중,NaN,NaN,NaN,NaN,NaN,...,38,0,101 401,"치과, 내과",3896.95,NaN,02-2600-5814,126.868313,37.549616,영업중
21,22,양천구보건소,서울특별시 양천구 신정동 321번지 5호,서울특별시 양천구 목동서로 339 (신정동),운영중,NaN,NaN,NaN,NaN,NaN,...,9,0,601,보건소,99999.00,NaN,02-2620-3919,126.865882,37.517619,영업중
22,23,서울특별시성동구보건소,서울특별시 성동구 홍익동 16번지 15호,서울특별시 성동구 마장로23길 10 (홍익동),운영중,NaN,NaN,NaN,NaN,NaN,...,8,0,601,보건소,3046.20,NaN,02-2286-7000,127.033087,37.567067,영업중
23,24,영등포구보건소,서울특별시 영등포구 당산동3가 385번지 1호,"서울특별시 영등포구 당산로 123 (당산동3가, 영등포구청)",운영중,NaN,NaN,NaN,NaN,NaN,...,9,0,101,내과,0.00,NaN,NaN,126.896275,37.526191,영업중
24,25,서울특별시송파구보건소,서울특별시 송파구 신천동 29번지 5호,서울특별시 송파구 올림픽로 326 (신천동),운영중,NaN,NaN,NaN,NaN,NaN,...,31,0,101 601,"보건소, 내과",1820.21,NaN,02-2147-3458,127.106067,37.514596,영업중


In [107]:
fp_test = 'custom.shp'
road_gdf = gpd.read_file(fp_test, encoding='cp949')
print('before len :', len(road_gdf.index))
road_gdf = road_gdf.dropna(subset=['geometry'])
print('after len :', len(road_gdf.index))
crs = road_gdf.crs
print(crs)

before len : 1360
after len : 1360
{'proj': 'tmerc', 'lat_0': 38, 'lon_0': 127.5, 'k': 0.9996, 'x_0': 1000000, 'y_0': 2000000, 'ellps': 'GRS80', 'units': 'm', 'no_defs': True}


In [500]:
from pyproj import Proj
#utm-k <-> wsg84 , inverse=True 옵션 키면 utm-k -> wsg84, Flase wsg84 -> utm-k
my_proj = Proj("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs +to +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
lat = [37.593920]
lon = [127.099437]
#utm_x, utm_y = my_proj(office_df['경도'].values, office_df['위도'].values, inverse=False)
utm_x, utm_y = my_proj(office_df['Y좌표(WGS)'].values, office_df['X좌표(WGS)'].values, inverse=False)
print(utm_x[0])
print(utm_y[0])
print(type(utm_x))
office_df['X'] = pd.Series(utm_x.round(0), index=office_df.index)
office_df['Y'] = pd.Series(utm_y.round(0), index=office_df.index)
office_df.head()

966252.2447910875
1943765.126418369
<class 'numpy.ndarray'>


,전철역명,호선,외부코드,사이버스테이션,X좌표,Y좌표,X좌표(WGS),Y좌표(WGS),X,Y,geometry
0,가락시장,8,817,2818,525992.0,1108579.0,37.492522,127.118234,966252.0,1943765.0,POINT (1e+30 1e+30)
1,가락시장,3,350,2818,525992.0,1108579.0,37.492522,127.118234,966252.0,1943765.0,POINT (1e+30 1e+30)
2,종로3가,5,534,153,498060.0,1130332.0,37.571607,126.991806,955123.0,1952592.0,POINT (1e+30 1e+30)
3,종로3가,3,329,153,498060.0,1130332.0,37.571607,126.991806,955123.0,1952592.0,POINT (1e+30 1e+30)
4,종로3가,1,130,153,498060.0,1130332.0,37.571607,126.991806,955123.0,1952592.0,POINT (1e+30 1e+30)


In [501]:
office_df['geometry'] = list(zip(office_df.X, office_df.Y))
office_df['geometry'] = office_df['geometry'].apply(Point)
office_df = office_df.iloc[0:, 1:]
pgdf = gpd.GeoDataFrame(office_df, crs=crs, geometry='geometry')
pgdf.head()

,호선,외부코드,사이버스테이션,X좌표,Y좌표,X좌표(WGS),Y좌표(WGS),X,Y,geometry
0,8,817,2818,525992.0,1108579.0,37.492522,127.118234,966252.0,1943765.0,POINT (966252 1943765)
1,3,350,2818,525992.0,1108579.0,37.492522,127.118234,966252.0,1943765.0,POINT (966252 1943765)
2,5,534,153,498060.0,1130332.0,37.571607,126.991806,955123.0,1952592.0,POINT (955123 1952592)
3,3,329,153,498060.0,1130332.0,37.571607,126.991806,955123.0,1952592.0,POINT (955123 1952592)
4,1,130,153,498060.0,1130332.0,37.571607,126.991806,955123.0,1952592.0,POINT (955123 1952592)


In [373]:
pgdf['지정고시일'] = pgdf['지정고시일'].apply(lambda x: str(x))

In [502]:
pgdf.to_file("subway.shp", encoding='cp949')
pgdf.crs

{'proj': 'tmerc',
 'lat_0': 38,
 'lon_0': 127.5,
 'k': 0.9996,
 'x_0': 1000000,
 'y_0': 2000000,
 'ellps': 'GRS80',
 'units': 'm',
 'no_defs': True,
 'wktext': True}

In [380]:
hdf = pd.read_excel('서울특별시 보건의료원(소) 정보_20180821.xlsx', encoding='cp-949')
print(hdf.shape)
hdf.tail(1)

(25, 23)


,번호,사업장명,소재지전체주소,도로명전체주소,영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,소재지면적,...,의료인수,입원실수,진료과목내용,진료과목내용명,총면적,총인원,전화번호,위치정보(X),위치정보(Y),상세영업상태명
24,25,서울특별시송파구보건소,서울특별시 송파구 신천동 29번지 5호,서울특별시 송파구 올림픽로 326 (신천동),운영중,NaN,NaN,NaN,NaN,NaN,...,31,0,101 601,"보건소, 내과",1820.21,NaN,02-2147-3458,209434.077592,446148.462945,영업중


In [383]:
from pyproj import Proj
#utm-k <-> wsg84 , inverse=True 옵션 키면 utm-k -> wsg84, Flase wsg84 -> utm-k
my_proj = Proj("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs +to +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
lat = [37.593920]
lon = [127.099437]
utm_x, utm_y = my_proj(hdf['위치정보(Y)'].values, hdf['위치정보(X)'].values, inverse=True)
print(utm_x[0])
print(utm_y[0])
print(type(utm_x))
hdf['X'] = pd.Series(utm_x.round(0), index=hdf.index)
hdf['Y'] = pd.Series(utm_y.round(0), index=hdf.index)
hdf.head(1)

122.17557935832546
21.73616414096759
<class 'numpy.ndarray'>


,번호,사업장명,소재지전체주소,도로명전체주소,영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,소재지면적,...,진료과목내용,진료과목내용명,총면적,총인원,전화번호,위치정보(X),위치정보(Y),상세영업상태명,X,Y
0,1,광진구보건소,자양동 777번지,서울특별시 광진구 자양로 117 (자양동),운영중,NaN,NaN,NaN,NaN,NaN,...,101,내과,0.0,NaN,450-1621,207312.705825,448818.375077,영업중,122.0,22.0


In [ ]:
hdf['geometry'] = list(zip(hdf.X, hdf.Y))
hdf['geometry'] = hdf['geometry'].apply(Point)
hdf = hdf.iloc[0:, 1:]
pgdf = gpd.GeoDataFrame(hdf, crs=crs, geometry='geometry')
pgdf.head()

In [132]:
bdf = pd.read_csv('build.csv')#, engine='python')

c:\programdata\anaconda3\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [133]:
print(bdf.shape)
bdf.tail()

(3916109, 19)


,관리_건축물대장_PK,관리_상위_건축물대장_PK,대장_구분_코드,대장_종류_코드,시군구_코드,법정동_코드,대지_구분_코드,특수지_명,블록,로트,위반_건축물_여부,대장_일련번호,총괄표제부_일련번호,표제부_일련번호,전유부_일련번호,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,작업_일자
3916104,11710-100477947,11710-100477591,2,4,11710,10800,0,NaN,NaN,NaN,0.0,373,1.0,1.0,373,1.171049e+11,10801.0,0.0,20190503
3916105,11710-100477541,11710-100477082,2,4,11710,10800,0,NaN,NaN,NaN,0.0,490,1.0,1.0,490,1.171049e+11,10801.0,0.0,20190307
3916106,11710-100477542,11710-100477082,2,4,11710,10800,0,NaN,NaN,NaN,0.0,491,1.0,1.0,491,1.171049e+11,10801.0,0.0,20190307
3916107,11710-100477543,11710-100477082,2,4,11710,10800,0,NaN,NaN,NaN,0.0,492,1.0,1.0,492,1.171049e+11,10801.0,0.0,20190307
3916108,11710-100477544,11710-100477082,2,4,11710,10800,0,NaN,NaN,NaN,0.0,493,1.0,1.0,493,1.171049e+11,10801.0,0.0,20190307


In [134]:
bdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3916109 entries, 0 to 3916108
Data columns (total 19 columns):
관리_건축물대장_PK       object
관리_상위_건축물대장_PK    object
대장_구분_코드          int64
대장_종류_코드          int64
시군구_코드            int64
법정동_코드            int64
대지_구분_코드          int64
특수지_명             object
블록                object
로트                object
위반_건축물_여부         float64
대장_일련번호           int64
총괄표제부_일련번호        float64
표제부_일련번호          float64
전유부_일련번호          int64
새주소_도로_코드         float64
새주소_법정동_코드        float64
새주소_지상지하_코드       float64
작업_일자             int64
dtypes: float64(6), int64(8), object(5)
memory usage: 567.7+ MB


In [141]:
fdf = bdf.loc[bdf['작업_일자'] < 20010101]

In [142]:
fdf.head()

,관리_건축물대장_PK,관리_상위_건축물대장_PK,대장_구분_코드,대장_종류_코드,시군구_코드,법정동_코드,대지_구분_코드,특수지_명,블록,로트,위반_건축물_여부,대장_일련번호,총괄표제부_일련번호,표제부_일련번호,전유부_일련번호,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,작업_일자


건축물대장 + 공시지가

In [240]:
rdf = pd.read_csv('build_info_filter.csv', encoding='cp949')
print(rdf.shape)

(237536, 57)


c:\programdata\anaconda3\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [242]:
rdf.head(1)

,관리건축물대장pk,대장구분코드,대장구분코드명,대장종류코드,대장종류코드명,구주소,도로명주소,건물명,시군구코드,법정동코드,...,부속건축물수,부속건축물면적,총동연면적,옥외자주식면적,허가일,착공일,사용승인일,생성일자,res,gosi
0,11380-15707,1,일반,2,일반건축물,서울특별시 은평구 갈현동 486-2번지,서울특별시 은평구 연서로21길 24-21,NaN,11380,10400.0,...,0,0.0,425.52,0.0,19910307,NaN,19910911.0,20120630,2,0


In [243]:
test = rdf.loc[rdf['사용승인일'] < 19900101]

In [244]:
len(test.index)

36769

In [274]:
fp_test = './shp/AL_11_D150_20190415.shp'
pdf = gpd.read_file(fp_test, encoding='cp949')
print('before len :', len(pdf.index))
pdf = pdf.dropna(subset=['geometry'])
print('after len :', len(pdf.index))
print(pdf.crs)

before len : 951061
after len : 951060
{'proj': 'tmerc', 'lat_0': 38, 'lon_0': 127.0028902777778, 'k': 1, 'x_0': 200000, 'y_0': 500000, 'ellps': 'bessel', 'units': 'm', 'no_defs': True}


In [276]:
pdf = pdf[['A0', 'A1', 'A2', 'A5', 'A9', 'geometry']]
pdf.head(1)

,A0,A1,A2,A5,A9,geometry
0,1138011400101630044,1138011400,서울특별시 은평구 진관동,163-44,0,"POLYGON ((193601.3310000002 458731.6999999993,..."


In [277]:
pdf['구주소'] = pdf.A2 + ' ' + pdf.A5 + '번지'

In [278]:
pdf.head()

,A0,A1,A2,A5,A9,geometry,구주소
0,1138011400101630044,1138011400,서울특별시 은평구 진관동,163-44,0,"POLYGON ((193601.3310000002 458731.6999999993,...",서울특별시 은평구 진관동 163-44번지
1,1138011400101630045,1138011400,서울특별시 은평구 진관동,163-45,0,"POLYGON ((193616.824 458749.1260000002, 193608...",서울특별시 은평구 진관동 163-45번지
2,1138011400101370036,1138011400,서울특별시 은평구 진관동,137-36,0,"POLYGON ((194427.3679999998 459455.7379999999,...",서울특별시 은평구 진관동 137-36번지
3,1138011400101370037,1138011400,서울특별시 은평구 진관동,137-37,0,"POLYGON ((194398.943 459478.1579999998, 194404...",서울특별시 은평구 진관동 137-37번지
4,1138011400101380020,1138011400,서울특별시 은평구 진관동,138-20,0,"POLYGON ((194287.8640000001 459462.0690000001,...",서울특별시 은평구 진관동 138-20번지


In [279]:
pdf = pdf.drop(['A2', 'A5'], axis=1)

In [280]:
pdf.head()

,A0,A1,A9,geometry,구주소
0,1138011400101630044,1138011400,0,"POLYGON ((193601.3310000002 458731.6999999993,...",서울특별시 은평구 진관동 163-44번지
1,1138011400101630045,1138011400,0,"POLYGON ((193616.824 458749.1260000002, 193608...",서울특별시 은평구 진관동 163-45번지
2,1138011400101370036,1138011400,0,"POLYGON ((194427.3679999998 459455.7379999999,...",서울특별시 은평구 진관동 137-36번지
3,1138011400101370037,1138011400,0,"POLYGON ((194398.943 459478.1579999998, 194404...",서울특별시 은평구 진관동 137-37번지
4,1138011400101380020,1138011400,0,"POLYGON ((194287.8640000001 459462.0690000001,...",서울특별시 은평구 진관동 138-20번지


건축물대장 필터링 한 건수 237536

In [353]:
bpdf = pd.merge(pdf, rdf, on='구주소', how='inner')
print(bpdf.shape)
bpdf.head(1)

(236579, 61)


,A0,A1,A9,geometry,구주소,관리건축물대장pk,대장구분코드,대장구분코드명,대장종류코드,대장종류코드명,...,부속건축물수,부속건축물면적,총동연면적,옥외자주식면적,허가일,착공일,사용승인일,생성일자,res,gosi
0,1159010200100220082,1159010200,2576000,"POLYGON ((195204.193 444823.7339999992, 195200...",서울특별시 동작구 상도동 22-82번지,11590-14903,1,일반,2,일반건축물,...,0,0.0,276.12,0.0,19900626,19901119,19901126.0,20170513,1,0


In [354]:
bpdf = bpdf.rename(columns={'A9': 'price'})

In [355]:
bpdf.head(1)

,A0,A1,price,geometry,구주소,관리건축물대장pk,대장구분코드,대장구분코드명,대장종류코드,대장종류코드명,...,부속건축물수,부속건축물면적,총동연면적,옥외자주식면적,허가일,착공일,사용승인일,생성일자,res,gosi
0,1159010200100220082,1159010200,2576000,"POLYGON ((195204.193 444823.7339999992, 195200...",서울특별시 동작구 상도동 22-82번지,11590-14903,1,일반,2,일반건축물,...,0,0.0,276.12,0.0,19900626,19901119,19901126.0,20170513,1,0


In [356]:
crs

{'proj': 'tmerc',
 'lat_0': 38,
 'lon_0': 127.5,
 'k': 0.9996,
 'x_0': 1000000,
 'y_0': 2000000,
 'ellps': 'GRS80',
 'units': 'm',
 'no_defs': True,
 'wktext': True}

In [357]:
bpdf['geometry'] = bpdf['geometry'].to_crs(crs)

In [358]:
bpdf.crs = crs

In [359]:
bpdf.crs

{'proj': 'tmerc',
 'lat_0': 38,
 'lon_0': 127.5,
 'k': 0.9996,
 'x_0': 1000000,
 'y_0': 2000000,
 'ellps': 'GRS80',
 'units': 'm',
 'no_defs': True,
 'wktext': True}

In [360]:
bpdf.to_file('home_price_all.shp', encoding='cp949')

In [361]:
fbpdf = bpdf.loc[(bpdf['사용승인일'] < 19900101) | (bpdf['gosi'] == 1)]

In [362]:
print(fbpdf.shape)
fbpdf.head(1)

(40475, 61)


,A0,A1,price,geometry,구주소,관리건축물대장pk,대장구분코드,대장구분코드명,대장종류코드,대장종류코드명,...,부속건축물수,부속건축물면적,총동연면적,옥외자주식면적,허가일,착공일,사용승인일,생성일자,res,gosi
1,1141011800100080685,1141011800,1784000,"POLYGON ((951601.7100766127 1956025.046085693,...",서울특별시 서대문구 홍은동 8-685번지,11410-18260,1,일반,2,일반건축물,...,0,0.0,66.12,0.0,NaN,NaN,19641120.0,20171020,1,0


In [363]:
fbpdf.to_file('home_price_filter.shp', encoding='cp949')

In [338]:
ht = pd.read_csv('old_hotel.csv', encoding='cp949')
print(ht.shape)
ht.head()

(1152, 9)


,Unnamed: 0,시군구코드,소재지지번,행정동명,year,old,area,room,room_area
0,3,3220000,서울특별시 강남구 역삼동 700번지 27호,역삼1동,1976,43,183.19,23,7.674583
1,5,3220000,서울특별시 강남구 논현동 56번지 21호,논현1동,1976,43,414.87,40,10.143171
2,32,3220000,서울특별시 강남구 논현동 6번지,논현1동,1979,40,344.45,122,2.808537
3,54,3220000,서울특별시 강남구 삼성동 49번지 9호,삼성2동,1980,39,256.70,21,11.713636
4,107,3220000,서울특별시 강남구 삼성동 100번지 3호,삼성1동,1983,36,309.01,20,14.762381


In [339]:
ht['소재지지번'] = ht['소재지지번'].apply(lambda x: x.replace('번지 ', '-'))
ht['소재지지번'] = ht['소재지지번'].apply(lambda x: x.strip())

In [340]:
ht['소재지지번'] = ht['소재지지번'].apply(lambda x: x.strip())
ht['소재지지번'] = ht['소재지지번'].apply(lambda x: x.replace('  ', ' '))
ht.head()

,Unnamed: 0,시군구코드,소재지지번,행정동명,year,old,area,room,room_area
0,3,3220000,서울특별시 강남구 역삼동 700-27호,역삼1동,1976,43,183.19,23,7.674583
1,5,3220000,서울특별시 강남구 논현동 56-21호,논현1동,1976,43,414.87,40,10.143171
2,32,3220000,서울특별시 강남구 논현동 6-,논현1동,1979,40,344.45,122,2.808537
3,54,3220000,서울특별시 강남구 삼성동 49-9호,삼성2동,1980,39,256.70,21,11.713636
4,107,3220000,서울특별시 강남구 삼성동 100-3호,삼성1동,1983,36,309.01,20,14.762381


In [341]:
ht['소재지지번'] = ht['소재지지번'].apply(lambda x: x[:-1])

In [342]:
ht.head()

,Unnamed: 0,시군구코드,소재지지번,행정동명,year,old,area,room,room_area
0,3,3220000,서울특별시 강남구 역삼동 700-27,역삼1동,1976,43,183.19,23,7.674583
1,5,3220000,서울특별시 강남구 논현동 56-21,논현1동,1976,43,414.87,40,10.143171
2,32,3220000,서울특별시 강남구 논현동 6,논현1동,1979,40,344.45,122,2.808537
3,54,3220000,서울특별시 강남구 삼성동 49-9,삼성2동,1980,39,256.70,21,11.713636
4,107,3220000,서울특별시 강남구 삼성동 100-3,삼성1동,1983,36,309.01,20,14.762381


In [343]:
ht['소재지지번'] = ht['소재지지번'] + '번지'

In [344]:
ht = ht.rename(columns={'소재지지번': '구주소'})
ht.head()

,Unnamed: 0,시군구코드,구주소,행정동명,year,old,area,room,room_area
0,3,3220000,서울특별시 강남구 역삼동 700-27번지,역삼1동,1976,43,183.19,23,7.674583
1,5,3220000,서울특별시 강남구 논현동 56-21번지,논현1동,1976,43,414.87,40,10.143171
2,32,3220000,서울특별시 강남구 논현동 6번지,논현1동,1979,40,344.45,122,2.808537
3,54,3220000,서울특별시 강남구 삼성동 49-9번지,삼성2동,1980,39,256.70,21,11.713636
4,107,3220000,서울특별시 강남구 삼성동 100-3번지,삼성1동,1983,36,309.01,20,14.762381


In [345]:
ht = ht.drop(['Unnamed: 0'], axis=1)

In [328]:
ht = ht[ht["구주소"].str.contains('-', na=False)]

In [329]:
ht.head()

,시군구코드,구주소,행정동명,year,old,area,room,room_area
0,3220000,서울특별시 강남구 역삼동 700-27번지,역삼1동,1976,43,183.19,23,7.674583
1,3220000,서울특별시 강남구 논현동 56-21번지,논현1동,1976,43,414.87,40,10.143171
3,3220000,서울특별시 강남구 삼성동 49-9번지,삼성2동,1980,39,256.70,21,11.713636
4,3220000,서울특별시 강남구 삼성동 100-3번지,삼성1동,1983,36,309.01,20,14.762381
5,3220000,서울특별시 강남구 도곡동 542-2번지,도곡1동,1984,35,522.49,37,13.776053


In [350]:
#bpdf = pd.merge(pdf, rdf, on='구주소', how='inner')
hgdf = pd.merge(pdf, ht, on='구주소', how='inner')
print(hgdf.shape)
hgdf.head()

(1015, 12)


,A0,A1,A9,geometry,구주소,시군구코드,행정동명,year,old,area,room,room_area
0,1111017500103520000,1111017500,2655000,"POLYGON ((201903.4050000003 452830.6280000005,...",서울특별시 종로구 숭인동 352번지,3000000,숭인제2동,1988,31,161.56,12,12.504615
1,1111017500103520000,1111017500,2655000,"POLYGON ((201903.4050000003 452830.6280000005,...",서울특별시 종로구 숭인동 352번지,3000000,숭인제2동,1988,31,161.56,12,12.504615
2,1111012800101920017,1111012800,10760000,"POLYGON ((198578.3609999996 452360.9210000001,...",서울특별시 종로구 관훈동 192-17번지,3000000,종로1.2.3.4가동,1984,35,178.01,26,6.630000
3,1111017400104640010,1111017400,6731000,"POLYGON ((200943.216 452122.2599999998, 200931...",서울특별시 종로구 창신동 464-10번지,3000000,창신제1동,1983,36,90.83,10,8.348182
4,1111017100101670001,1111017100,4314000,"POLYGON ((199853.04 453512.9199999999, 199848....",서울특별시 종로구 명륜2가 167-1번지,3000000,혜화동,1979,40,111.47,10,10.224545


In [351]:
hgdf.to_file('old_hotel.shp', encoding='cp949')

In [352]:
print(pdf.shape)
pdf.head(1)

(946548, 5)


,A0,A1,A9,geometry,구주소
0,1138011400101630044,1138011400,0,"POLYGON ((193601.3310000002 458731.6999999993,...",서울특별시 은평구 진관동 163-44번지


In [331]:
pdf.loc[pdf['구주소'] == '서울특별시 강남구 역삼동 700-27번지']

,A0,A1,A9,geometry,구주소
874284,1168010100107000027,1168010100,13570000,"POLYGON ((203769.4500000002 444711.5399999991,...",서울특별시 강남구 역삼동 700-27번지


In [334]:
if pdf.iloc[874284]['구주소'] == ht.iloc[0]['구주소']:
    print('a')
else:
    print('b')

b


In [336]:
print(pdf.iloc[874284]['구주소'])
print(ht.iloc[0]['구주소'])

서울특별시 강남구 역삼동 836-35번지
서울특별시 강남구 역삼동  700-27번지


In [348]:
pdf = pdf.drop_duplicates(subset=['구주소'], keep='first')
print(pdf.shape)

(946548, 5)


In [317]:
pdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 951060 entries, 0 to 951060
Data columns (total 5 columns):
A0          951060 non-null object
A1          951060 non-null object
A9          951060 non-null int64
geometry    951060 non-null object
구주소         951058 non-null object
dtypes: int64(1), object(4)
memory usage: 43.5+ MB


In [332]:
ht.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1078 entries, 0 to 1151
Data columns (total 8 columns):
시군구코드        1078 non-null int64
구주소          1078 non-null object
행정동명         1078 non-null object
year         1078 non-null int64
old          1078 non-null int64
area         1078 non-null float64
room         1078 non-null int64
room_area    1078 non-null float64
dtypes: float64(2), int64(4), object(2)
memory usage: 75.8+ KB


In [225]:
fp_test = 'target_time3_home_pop_w8.shp'
ccc = gpd.read_file(fp_test, encoding='cp949')
print('before len :', len(ccc.index))
ccc = ccc.dropna(subset=['geometry'])
print('after len :', len(ccc.index))
print(ccc.crs)

before len : 320
after len : 320
{}


In [228]:
target_df.head()

,TOT_REG_CD,ADM_NM,ADM_CD,geometry,시간대구분,집계구코드,총생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수,sum,seq,취약독거노인비,fin
113,1101061010001,종로1·2·3·4가동,1101061,"POLYGON ((954918.3890857697 1954371.537916183,...",3,1101061010001,1555.80,25.65,61.35,87.00,113,0.391408,34.052506
114,1101061020002,종로1·2·3·4가동,1101061,"POLYGON ((955615.6320858002 1953061.522615433,...",3,1101061020002,889.75,19.00,23.65,42.65,114,0.391408,16.693556
115,1101061020001,종로1·2·3·4가동,1101061,"POLYGON ((955528.0457859039 1952865.453016281,...",3,1101061020001,2327.05,60.90,106.85,167.75,115,0.391408,65.658711
116,1101061020301,종로1·2·3·4가동,1101061,"POLYGON ((955645.9566841139 1952542.829115113,...",3,1101061020301,149.05,4.30,1.40,5.70,116,0.391408,2.231026
117,1101061030007,종로1·2·3·4가동,1101061,"POLYGON ((955020.3065853119 1953041.623815536,...",3,1101061030007,656.85,17.45,29.60,47.05,117,0.391408,18.415752


In [229]:
target_df['geometry'] = target_df['geometry'].centroid

In [230]:
target_df.head()

,TOT_REG_CD,ADM_NM,ADM_CD,geometry,시간대구분,집계구코드,총생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수,sum,seq,취약독거노인비,fin
113,1101061010001,종로1·2·3·4가동,1101061,POINT (955209.4255837937 1953717.510217959),3,1101061010001,1555.80,25.65,61.35,87.00,113,0.391408,34.052506
114,1101061020002,종로1·2·3·4가동,1101061,POINT (955622.8051095132 1952955.417046088),3,1101061020002,889.75,19.00,23.65,42.65,114,0.391408,16.693556
115,1101061020001,종로1·2·3·4가동,1101061,POINT (955347.9228788523 1952824.112416267),3,1101061020001,2327.05,60.90,106.85,167.75,115,0.391408,65.658711
116,1101061020301,종로1·2·3·4가동,1101061,POINT (955713.9435323275 1952571.432016511),3,1101061020301,149.05,4.30,1.40,5.70,116,0.391408,2.231026
117,1101061030007,종로1·2·3·4가동,1101061,POINT (954970.1366068629 1952968.387704964),3,1101061030007,656.85,17.45,29.60,47.05,117,0.391408,18.415752


In [231]:
target_df['xx'] = target_df['geometry'].apply(lambda x: x.x)
target_df['yy'] = target_df['geometry'].apply(lambda x: x.y)

In [232]:
target_df.head()

,TOT_REG_CD,ADM_NM,ADM_CD,geometry,시간대구분,집계구코드,총생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수,sum,seq,취약독거노인비,fin,xx,yy
113,1101061010001,종로1·2·3·4가동,1101061,POINT (955209.4255837937 1953717.510217959),3,1101061010001,1555.80,25.65,61.35,87.00,113,0.391408,34.052506,955209.425584,1.953718e+06
114,1101061020002,종로1·2·3·4가동,1101061,POINT (955622.8051095132 1952955.417046088),3,1101061020002,889.75,19.00,23.65,42.65,114,0.391408,16.693556,955622.805110,1.952955e+06
115,1101061020001,종로1·2·3·4가동,1101061,POINT (955347.9228788523 1952824.112416267),3,1101061020001,2327.05,60.90,106.85,167.75,115,0.391408,65.658711,955347.922879,1.952824e+06
116,1101061020301,종로1·2·3·4가동,1101061,POINT (955713.9435323275 1952571.432016511),3,1101061020301,149.05,4.30,1.40,5.70,116,0.391408,2.231026,955713.943532,1.952571e+06
117,1101061030007,종로1·2·3·4가동,1101061,POINT (954970.1366068629 1952968.387704964),3,1101061030007,656.85,17.45,29.60,47.05,117,0.391408,18.415752,954970.136607,1.952968e+06


In [235]:
from pyproj import Proj
#utm-k <-> wsg84 , inverse=True 옵션 키면 utm-k -> wsg84, Flase wsg84 -> utm-k
my_proj = Proj("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs +to +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
lat = [37.593920]
lon = [127.099437]
utm_x, utm_y = my_proj(target_df['xx'].values, target_df['yy'].values, inverse=True)
print(utm_x[0])
print(utm_y[0])
print(type(utm_x))
target_df['X'] = pd.Series(utm_x, index=target_df.index)
target_df['Y'] = pd.Series(utm_y, index=target_df.index)
target_df.head()

126.99271050277724
37.58175567915511
<class 'numpy.ndarray'>


,TOT_REG_CD,ADM_NM,ADM_CD,geometry,시간대구분,집계구코드,총생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수,sum,seq,취약독거노인비,fin,xx,yy,X,Y
113,1101061010001,종로1·2·3·4가동,1101061,POINT (955209.4255837937 1953717.510217959),3,1101061010001,1555.80,25.65,61.35,87.00,113,0.391408,34.052506,955209.425584,1.953718e+06,126.992711,37.581756
114,1101061020002,종로1·2·3·4가동,1101061,POINT (955622.8051095132 1952955.417046088),3,1101061020002,889.75,19.00,23.65,42.65,114,0.391408,16.693556,955622.805110,1.952955e+06,126.997438,37.574907
115,1101061020001,종로1·2·3·4가동,1101061,POINT (955347.9228788523 1952824.112416267),3,1101061020001,2327.05,60.90,106.85,167.75,115,0.391408,65.658711,955347.922879,1.952824e+06,126.994333,37.573710
116,1101061020301,종로1·2·3·4가동,1101061,POINT (955713.9435323275 1952571.432016511),3,1101061020301,149.05,4.30,1.40,5.70,116,0.391408,2.231026,955713.943532,1.952571e+06,126.998494,37.571450
117,1101061030007,종로1·2·3·4가동,1101061,POINT (954970.1366068629 1952968.387704964),3,1101061030007,656.85,17.45,29.60,47.05,117,0.391408,18.415752,954970.136607,1.952968e+06,126.990047,37.574992


In [237]:
target_df = target_df.drop(['seq', 'xx', 'yy'], axis=1)

In [238]:
target_df.head()

,TOT_REG_CD,ADM_NM,ADM_CD,geometry,시간대구분,집계구코드,총생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수,sum,취약독거노인비,fin,X,Y
113,1101061010001,종로1·2·3·4가동,1101061,POINT (955209.4255837937 1953717.510217959),3,1101061010001,1555.80,25.65,61.35,87.00,0.391408,34.052506,126.992711,37.581756
114,1101061020002,종로1·2·3·4가동,1101061,POINT (955622.8051095132 1952955.417046088),3,1101061020002,889.75,19.00,23.65,42.65,0.391408,16.693556,126.997438,37.574907
115,1101061020001,종로1·2·3·4가동,1101061,POINT (955347.9228788523 1952824.112416267),3,1101061020001,2327.05,60.90,106.85,167.75,0.391408,65.658711,126.994333,37.573710
116,1101061020301,종로1·2·3·4가동,1101061,POINT (955713.9435323275 1952571.432016511),3,1101061020301,149.05,4.30,1.40,5.70,0.391408,2.231026,126.998494,37.571450
117,1101061030007,종로1·2·3·4가동,1101061,POINT (954970.1366068629 1952968.387704964),3,1101061030007,656.85,17.45,29.60,47.05,0.391408,18.415752,126.990047,37.574992


In [239]:
target_df.to_csv('time_3_pop_xy.csv', encoding='cp949', index=False)

In [365]:
bpdf.iloc[0]

A0                                         1159010200100220082
A1                                                  1159010200
price                                                  2576000
geometry     POLYGON ((951267.4248071301 1944981.264513797,...
구주소                                      서울특별시 동작구 상도동 22-82번지
관리건축물대장pk                                          11590-14903
대장구분코드                                                       1
대장구분코드명                                                     일반
대장종류코드                                                       2
대장종류코드명                                                  일반건축물
도로명주소                                        서울특별시 동작구 만양로 7-5
건물명                                                        NaN
시군구코드                                                    11590
법정동코드                                                    10200
대지구분코드                                                       0
번                                                      

대상 행정동과 취약 주택 정보 조인

In [392]:
intersections = gpd.sjoin(fbpdf, target_df, how="inner", op='intersects')

c:\programdata\anaconda3\envs\geo\lib\site-packages\geopandas\tools\sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')


In [393]:
len(intersections.index)

927

In [394]:
intersections.head(1)

,A0,A1,price,geometry,구주소,관리건축물대장pk,대장구분코드,대장구분코드명,대장종류코드,대장종류코드명,...,ADM_CD,시간대구분,집계구코드,총생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수,sum,seq,취약독거노인비,fin
374,1111017500113750000,1111017500,7230000,"POLYGON ((958018.0292359968 1952588.626716308,...",서울특별시 종로구 숭인동 1375번지,11110-15528,1,일반,2,일반건축물,...,1106081,3,1106081120001,1111.25,20.0,33.6,53.6,16742,0.575216,30.831585


In [395]:
intersections['in_target'] = 1
intersections.head()

,A0,A1,price,geometry,구주소,관리건축물대장pk,대장구분코드,대장구분코드명,대장종류코드,대장종류코드명,...,시간대구분,집계구코드,총생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수,sum,seq,취약독거노인비,fin,in_target
374,1111017500113750000,1111017500,7230000,"POLYGON ((958018.0292359968 1952588.626716308,...",서울특별시 종로구 숭인동 1375번지,11110-15528,1,일반,2,일반건축물,...,3,1106081120001,1111.25,20.0,33.6,53.6,16742,0.575216,30.831585,1
25655,1123010100100980037,1123010100,3478000,"POLYGON ((958332.8258781206 1952721.232688003,...",서울특별시 동대문구 신설동 98-37번지,11230-100229492,1,일반,2,일반건축물,...,3,1106081120001,1111.25,20.0,33.6,53.6,16742,0.575216,30.831585,1
28541,1123010100101040003,1123010100,4132000,"POLYGON ((958191.9421358835 1952562.449141755,...",서울특별시 동대문구 신설동 104-3번지,11230-25102,1,일반,2,일반건축물,...,3,1106081120001,1111.25,20.0,33.6,53.6,16742,0.575216,30.831585,1
28550,1123010100101030001,1123010100,8330000,"POLYGON ((958191.6727062595 1952625.850236216,...",서울특별시 동대문구 신설동 103-1번지,11230-32634,1,일반,2,일반건축물,...,3,1106081120001,1111.25,20.0,33.6,53.6,16742,0.575216,30.831585,1
30000,1123010100100980012,1123010100,5069000,"POLYGON ((958311.351871103 1952692.836646006, ...",서울특별시 동대문구 신설동 98-12번지,11230-25954,1,일반,2,일반건축물,...,3,1106081120001,1111.25,20.0,33.6,53.6,16742,0.575216,30.831585,1


In [396]:
check_df = intersections[['구주소', 'in_target']]
check_df.head()

,구주소,in_target
374,서울특별시 종로구 숭인동 1375번지,1
25655,서울특별시 동대문구 신설동 98-37번지,1
28541,서울특별시 동대문구 신설동 104-3번지,1
28550,서울특별시 동대문구 신설동 103-1번지,1
30000,서울특별시 동대문구 신설동 98-12번지,1


In [397]:
mdf = pd.merge(fbpdf, check_df, on='구주소', how='left')

In [400]:
mdf['in_target'].fillna(0,  inplace=True)

In [405]:
mdf.to_csv('train.csv', encoding='cp949', index=False)

In [404]:
print(len(mdf.index))
mdf = mdf.drop_duplicates(subset=['구주소'], keep='first')
print(len(mdf.index))

40653
40475


In [406]:
mdf['geometry'] = mdf['geometry'].centroid

c:\programdata\anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [407]:
mdf.head(1)

,A0,A1,price,geometry,구주소,관리건축물대장pk,대장구분코드,대장구분코드명,대장종류코드,대장종류코드명,...,부속건축물면적,총동연면적,옥외자주식면적,허가일,착공일,사용승인일,생성일자,res,gosi,in_target
0,1141011800100080685,1141011800,1784000,POINT (951595.8959097113 1956016.413927333),서울특별시 서대문구 홍은동 8-685번지,11410-18260,1,일반,2,일반건축물,...,0.0,66.12,0.0,NaN,NaN,19641120.0,20171020,1,0,0.0


In [503]:
fp_test = 'subway.shp'
point_gdf = gpd.read_file(fp_test, encoding='cp949')
print('before len :', len(point_gdf.index))
point_gdf = point_gdf.dropna(subset=['geometry'])
print('after len :', len(point_gdf.index))
print(point_gdf.crs)

before len : 929
after len : 929
{'proj': 'tmerc', 'lat_0': 38, 'lon_0': 127.5, 'k': 0.9996, 'x_0': 1000000, 'y_0': 2000000, 'ellps': 'GRS80', 'units': 'm', 'no_defs': True}


In [504]:
mp = MultiPoint(list(point_gdf.geometry.values))
len(mp)

929

In [505]:
def make_dist(row, xy_point):
    #xy_point = Point(x, y)
    res = nearest_points(xy_point, mp)
    dist = xy_point.distance(res[1])
    return dist

mdf['subway_dist'] = mdf.apply(lambda x:make_dist(x, x['geometry']), axis=1)    

In [506]:
mdf.head(2)

,A0,A1,price,geometry,구주소,관리건축물대장pk,대장구분코드,대장구분코드명,대장종류코드,대장종류코드명,...,생성일자,res,gosi,in_target,office_dist,park_dist,market_dist,welfare_dist,public_hospital_dist,subway_dist
0,1141011800100080685,1141011800,1784000,POINT (951595.8959097113 1956016.413927333),서울특별시 서대문구 홍은동 8-685번지,11410-18260,1,일반,2,일반건축물,...,20171020,1,0,0.0,707.505064,1023.975805,1440.152805,3036.598402,2010.583712,1410.207250
1,1111014000100260007,1111014000,4655000,POINT (954451.4392070894 1954113.482411152),서울특별시 종로구 삼청동 26-7번지,11110-19412,1,일반,2,일반건축물,...,20171118,1,0,0.0,910.728245,614.250302,863.505717,1047.783288,1369.082615,985.050193


In [507]:
mdf.to_csv('train_final_sw.csv', encoding='cp949', index=False)

In [508]:
pred = pd.read_excel('pred_final_sw.xlsx', encoding='cp949')

In [509]:
pred.head(1)

,geometry,시군구코드,구주소,price,office_dist,park_dist,market_dist,welfare_dist,subway_dist,in_target,office_log,market_log,welfare_log,subway_log,price_sqrt,pred
0,POINT (951595.8959097113 1956016.413927333),11410,서울특별시 서대문구 홍은동 8-685번지,1784000,707.505064,1023.975805,1440.152805,3036.598402,1410.20725,0,6.563157,7.273199,8.018822,7.252201,1335.665003,0


In [510]:
pred.shape

(39726, 16)

In [511]:
pf = pred.loc[pred['pred'] == 1]

In [512]:
pf.shape

(7265, 16)

In [513]:
pf['geometry'] = pf['geometry'].apply(lambda x: x.replace('POINT (', '').replace(')', '').strip())

c:\programdata\anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [514]:
pf['X'] = pf['geometry'].apply(lambda x: x.split(' ')[0])

c:\programdata\anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [515]:
pf['Y'] = pf['geometry'].apply(lambda x: x.split(' ')[1])

c:\programdata\anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [516]:
pf.head(1)

,geometry,시군구코드,구주소,price,office_dist,park_dist,market_dist,welfare_dist,subway_dist,in_target,office_log,market_log,welfare_log,subway_log,price_sqrt,pred,X,Y
2,952942.5699143736 1953214.75949659,11110,서울특별시 종로구 누상동 166-200번지,3322000,522.486924,119.300898,476.7993,1233.081022,589.564319,0,6.260512,6.169191,7.118082,6.381079,1822.635729,1,952942.5699143736,1953214.75949659


In [517]:
pf['X'] = pd.to_numeric(pf['X'])
pf['Y'] = pd.to_numeric(pf['Y'])

c:\programdata\anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\programdata\anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [518]:
pf['geometry'] = list(zip(pf.X, pf.Y))
pf['geometry'] = pf['geometry'].apply(Point)

c:\programdata\anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\programdata\anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [519]:
print('before len :', len(pf.index))
pf = pf.dropna(subset=['geometry'])
print('after len :', len(pf.index))


before len : 7265
after len : 7265


In [520]:
aapf = gpd.GeoDataFrame(pf, crs=crs, geometry='geometry')
aapf.head()

,geometry,시군구코드,구주소,price,office_dist,park_dist,market_dist,welfare_dist,subway_dist,in_target,office_log,market_log,welfare_log,subway_log,price_sqrt,pred,X,Y
2,POINT (952942.5699143736 1953214.75949659),11110,서울특별시 종로구 누상동 166-200번지,3322000,522.486924,119.300898,476.799300,1233.081022,589.564319,0,6.260512,6.169191,7.118082,6.381079,1822.635729,1,952942.569914,1.953215e+06
8,POINT (952911.9144122802 1953216.27596912),11110,서울특별시 종로구 누상동 166-251번지,3387000,507.542293,142.117377,492.601248,1216.931218,619.599699,0,6.231548,6.201728,7.104909,6.430686,1840.380667,1,952911.914412,1.953216e+06
9,POINT (954382.0062039712 1953625.20754148),11110,서울특별시 종로구 삼청동 147-11번지,6530000,564.519914,517.213170,370.332248,590.154423,523.249618,0,6.337746,5.917097,6.382077,6.261968,2555.386664,1,954382.006204,1.953625e+06
16,POINT (953297.267618528 1952459.907818367),11110,서울특별시 종로구 신문로2가 1-138번지,5936000,90.096524,53.301773,891.947659,1094.877830,494.467569,0,4.511920,6.794528,6.999311,6.205502,2436.390978,1,953297.267619,1.952460e+06
21,POINT (956037.4298511582 1952374.498203163),11110,서울특별시 종로구 연지동 239-2번지,4846000,120.925290,528.998221,180.765356,843.224363,140.211058,0,4.803408,5.202717,6.738418,4.950256,2201.363441,1,956037.429851,1.952374e+06


In [521]:
aapf.to_file('pred_sw.shp', encoding='cp949')